# 导入部分

In [1]:
import torch
import pandas as pd
import numpy as np
import argparse
# tqdm是进度条库 将range改为trange即可生效
# from tqdm import tqdm, trange
import torch.nn as nn
import my_model
import my_data

# 辅助动图类

In [2]:
%matplotlib inline
from IPython import display
from matplotlib import pyplot as plt
from matplotlib_inline import backend_inline

class Animator:
    """在动画中绘制数据"""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # 增量地绘制多条线
        if legend is None:
            legend = []
        backend_inline.set_matplotlib_formats('svg')
        self.fig, self.axes = plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # 使用lambda函数捕获参数
        self.config_axes = lambda: self.set_axes(xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def set_axes(self, xlabel, ylabel, xlim, ylim, xscale, yscale, legend):
        """设置matplotlib的轴"""
        self.axes[0].set_xlabel(xlabel)
        self.axes[0].set_ylabel(ylabel)
        self.axes[0].set_xscale(xscale)
        self.axes[0].set_yscale(yscale)
        self.axes[0].set_xlim(xlim)
        self.axes[0].set_ylim(ylim)
        if legend:
            self.axes[0].legend(legend)
        self.axes[0].grid()


    def add(self, x, y):
        # 向图表中添加多个数据点
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        plt.draw()
        plt.pause(0.001)
        display.display(self.fig)
        display.clear_output(wait=True)
        
    def show(self):
        display.display(self.fig)


# 数据处理和评估函数

In [3]:
def process_data(file_path, edge_path):
    # 读取数据
    df = pd.read_csv(file_path, encoding='utf-8')
    edge_df = pd.read_csv(edge_path, encoding='utf-8')

    # 初始化字典和计数器
    geohash_df_dict = {}
    date_df_dict = {}
    number_hash = 0
    number_date = 0

    # 为geohash_id创建映射
    for i in df["geohash_id"]:
        if i not in geohash_df_dict.keys():
            geohash_df_dict[i] = number_hash
            number_hash += 1

    # 为date_id创建映射
    for i in df["date_id"]:
        if i not in date_df_dict.keys():
            date_df_dict[i] = number_date
            number_date += 1

    # 初始化新数据
    new_data = np.zeros((len(date_df_dict), len(geohash_df_dict), 1 + len(df.iloc[0, 2:])))

    # 填充新数据
    for index, row in df.iterrows():
        hash_index, date_index = geohash_df_dict[row["geohash_id"]], date_df_dict[row["date_id"]]
        # 将date_index和df.iloc[0, 2:]填充到new_data[date_index][hash_index]位置中
        new_data[date_index][hash_index] = np.hstack((date_index, np.array(df.iloc[index, 2:])))
    
    np.save("npdata/new_data.npy", new_data)

    # 构建邻接矩阵
    x_mask = np.zeros((len(date_df_dict), len(geohash_df_dict), len(geohash_df_dict), 1), dtype=float)
    x_edge_df = np.zeros((len(date_df_dict), len(geohash_df_dict), len(geohash_df_dict), 2), dtype=float)

    # 填充邻接矩阵
    for index, row in edge_df.iterrows():
        if row["geohash6_point1"] not in geohash_df_dict.keys() or row["geohash6_point2"] not in geohash_df_dict.keys():
            continue
        point1_index, point2_index, F_1, F_2, date_index = geohash_df_dict[row["geohash6_point1"]], geohash_df_dict[row["geohash6_point2"]], row["F_1"], row["F_2"], date_df_dict[row["date_id"]]
        x_mask[date_index][point1_index][point2_index] = 1
        x_mask[date_index][point2_index][point1_index] = 1
        x_edge_df[date_index][point1_index][point2_index] = [F_1, F_2]
        x_edge_df[date_index][point2_index][point1_index] = [F_1, F_2]
    
    np.save("npdata/x_mask.npy", x_mask)
    np.save("npdata/x_edge_df.npy", x_edge_df)
    
    return geohash_df_dict, number_date

In [4]:
def process_test_data(file_path, edge_path, pre_geohash_df_dict, pre_number_date):
    # 读取数据
    df = pd.read_csv(file_path, encoding='utf-8')
    edge_df = pd.read_csv(edge_path, encoding='utf-8')

    # 初始化字典和计数器
    geohash_df_dict = pre_geohash_df_dict
    date_df_dict = {}
    number_date = pre_number_date
    
    # 为date_id创建映射
    for i in df["date_id"]:
        if i not in date_df_dict.keys():
            date_df_dict[i] = number_date
            number_date += 1

    # 初始化新数据
    new_data = np.zeros((len(date_df_dict), len(geohash_df_dict), 1 + len(df.iloc[0, 2:])))

    # 填充新数据
    for index, row in df.iterrows():
        hash_index, date_index = geohash_df_dict[row["geohash_id"]], date_df_dict[row["date_id"]]
        # 将date_index和df.iloc[0, 2:]填充到new_data[date_index][hash_index]位置中
        new_data[date_index - pre_number_date][hash_index] = np.hstack((date_index, np.array(df.iloc[index, 2:])))
    
    np.save("test_npdata/new_data.npy", new_data)

    # 构建邻接矩阵
    x_mask = np.zeros((len(date_df_dict), len(geohash_df_dict), len(geohash_df_dict), 1), dtype=float)
    x_edge_df = np.zeros((len(date_df_dict), len(geohash_df_dict), len(geohash_df_dict), 2), dtype=float)

    # 填充邻接矩阵
    for index, row in edge_df.iterrows():
        if row["geohash6_point1"] not in geohash_df_dict.keys() or row["geohash6_point2"] not in geohash_df_dict.keys():
            continue
        point1_index, point2_index, F_1, F_2, date_index = geohash_df_dict[row["geohash6_point1"]], geohash_df_dict[row["geohash6_point2"]], row["F_1"], row["F_2"], date_df_dict[row["date_id"]]
        x_mask[date_index - pre_number_date][point1_index][point2_index] = 1
        x_mask[date_index - pre_number_date][point2_index][point1_index] = 1
        x_edge_df[date_index - pre_number_date][point1_index][point2_index] = [F_1, F_2]
        x_edge_df[date_index - pre_number_date][point2_index][point1_index] = [F_1, F_2]

    np.save("test_npdata/x_mask.npy", x_mask)
    np.save("test_npdata/x_edge_df.npy", x_edge_df)

In [16]:
criterion = nn.MSELoss()

def eval(model, dataset, args):
    model.eval()
    with torch.no_grad():
        dev_loss = []
        for j in range(dataset.batch_count):
            x_date, x_feature, x_mask_data, x_edge_data, x_tags = dataset.get_batch(j)
            act_pre, con_pre = model(x_date, x_feature, x_mask_data)
            predict = torch.cat((act_pre, con_pre), dim=-1)
            loss = criterion(predict, x_tags)
            dev_loss.append(loss)
        model.train()
        dev_loss = sum(dev_loss) / len(dev_loss)
    return dev_loss.detach().cpu()


# 训练函数

In [18]:
def train(args, x_train, x_mask, x_edge_df):
    
    if args.rat != 1.0:
        # 根据参数比例分割训练集和验证集，并转换为张量
        x_train, x_dev = torch.tensor(x_train[:int(len(x_train)*args.rat)]), torch.tensor(x_train[int(len(x_train)*args.rat):])
        x_mask_train, x_mask_dev = torch.tensor(x_mask[:int(len(x_mask)*args.rat)]), torch.tensor(x_mask[int(len(x_mask)*args.rat):])
        x_edge_train, x_edge_dev = torch.FloatTensor(x_edge_df[:int(len(x_edge_df) * args.rat)]), torch.tensor(x_edge_df[int(len(x_edge_df) * args.rat):])
    else:
        x_train = torch.tensor(x_train)
        x_mask_train = torch.tensor(x_mask)
        x_edge_train = torch.tensor(x_edge_df)
    # 设置日期嵌入的维度
    date_emb = 5
    
    # 初始化模型
    # len(date_df_dict)恒为90
    model = my_model.GAT(date_emb=[90, date_emb], nfeat=35, nhid=40, dropout=0.3, alpha=0.2, nheads=10).to(args.device)

    # 设置优化器
    optimizer = torch.optim.Adam(params=model.parameters(), lr=args.lr)

    # 设置模型为训练模式
    model.train()

    # 创建训练集和验证集的迭代器
    trainset = my_data.DataIterator(x_train, x_mask_train, x_edge_train, args)
    if args.rat != 1.0:
        valset = my_data.DataIterator(x_dev, x_mask_dev, x_edge_dev, args)

        animator = Animator(xlabel='epoch', xlim=[1, args.epochs], ylim=None,
                        legend=['train loss', 'dev loss'])

    # 进行多轮训练
    for indx in range(args.epochs):
        train_all_loss = []

        # 对每个批次进行训练
        for j in range(trainset.batch_count):
            x_date, x_feature, x_mask_data, x_edge_data, x_tags = trainset.get_batch(j)
            act_pre, con_pre = model(x_date, x_feature, x_mask_data)
            predict = torch.cat((act_pre, con_pre), dim=-1)

            # 计算损失
            loss = criterion(predict, x_tags)
            train_all_loss.append(loss)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if args.rat != 1.0:
            
            # 绘制训练损失曲线
            train_loss = sum(train_all_loss) / len(train_all_loss)
            train_loss = train_loss.detach().cpu()
            
            # 对验证集进行评估
            dev_loss = eval(model, valset, args)
            
            # 对val_loss开根号
            sqrt_dev_loss = torch.sqrt(torch.tensor(dev_loss * 4))
            
            # 计算score
            score = 1 / (1 + sqrt_dev_loss)
            
            # 打印损失
            print('Epoch: {0}, Train loss: {1}, Dev loss: {2}'.format(indx + 1, train_loss, dev_loss))
            
            # 打印score
            print('Epoch: {0}, Score: {1}'.format(indx + 1, score))
            
            # 动态绘制验证损失曲线
            animator.add(indx + 1, (train_loss, dev_loss))
        else:
            
            train_loss = train_all_loss / trainset.batch_count
            
            sqet_train_loss = torch.sqrt(torch.tensor(train_loss * 4 * 1140))
            
            # 计算score
            score = 1 / (1 + sqet_train_loss)
            
            # 打印损失
            print('Epoch: {0}, Train loss: {1}'.format(indx + 1, train_loss))
            
            # 打印score
            print('Epoch: {0}, Score: {1}'.format(indx + 1, score))
            
    
    # 返回模型
    return model

In [7]:
def predict(model, x_test, x_mask_test, x_edge_test):
    
    print("x_test.shape: ", x_test.shape)
    print("x_mask_test.shape: ", x_mask_test.shape)
    print("x_edge_test.shape: ", x_edge_test.shape)
    
    # 转换为张量
    x_test, x_mask_test, x_edge_test = torch.FloatTensor(x_test), torch.tensor(x_mask_test), torch.tensor(x_edge_test)
    
    # 如果设备不匹配，使用to()方法移动张量
    if next(model.parameters()).device != x_test.device:
        x_test = x_test.to(next(model.parameters()).device)
        x_mask_test = x_mask_test.to(next(model.parameters()).device)
        x_edge_test = x_edge_test.to(next(model.parameters()).device)
    
    # 预测
    model.eval()
    with torch.no_grad():
        act_pre, con_pre = model(x_test[:,:,:1], x_test[:,:,1:], x_mask_test) # 没有用到边数据，所以用两个x_test占位
        # 将两个预测结果作为两列并输出为csv文件，并使用当前时间进行标注
        predict = torch.cat((act_pre, con_pre), dim=-1)
        predict = predict.detach().cpu().numpy()
        # numpy数组中的第一维是日期，将数据按第一维展开，使得数据变为二维
        predict = predict.reshape(-1, predict.shape[-1])
        total_predict = pd.DataFrame(predict)
        total_predict.to_csv("prediction/total_predict.csv", index=False, header=False)
        # 保留最后4*1140行，即保留预测结果
        predict = predict[-4*1140:]
        predict = pd.DataFrame(predict)
        predict.to_csv("prediction/predict.csv", index=False, header=False)
        

# 执行部分

In [8]:
# 执行数据处理，只需要执行一次，后面只需要读取数据即可
# geohash_df_dict, number_date = process_data("data/train_90.csv", "data/edge_90.csv")

In [9]:
# 读取数据
x_train = np.load("train_npdata/new_data.npy")
x_mask = np.load("train_npdata/x_mask.npy")
x_edge_df = np.load("train_npdata/x_edge_df.npy")

print("x_train.shape: ", x_train.shape)
print("x_mask.shape: ", x_mask.shape)
print("x_edge_df.shape: ", x_edge_df.shape)

x_train.shape:  (90, 1140, 38)
x_mask.shape:  (90, 1140, 1140, 1)
x_edge_df.shape:  (90, 1140, 1140, 2)


In [10]:
# 用于检验原来数据处理代码中的bug
for i in range(90):
    for j in range(1140):
        if x_train[i][j][0] != 89.0:
            print('xxx')

xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx
xxx


In [11]:
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=300,
                    help='training epoch number')
parser.add_argument('--batch_size', type=int, default=4,
                    help='batch_size')
parser.add_argument('--device', type=str, default="cuda",
                    help='gpu or cpu')
parser.add_argument('--lr', type=float, default=1e-3,
                    )
parser.add_argument('--rat', type=float, default=0.9,)

parser.add_argument('--decline', type=int, default=30, help="number of epochs to decline")

_StoreAction(option_strings=['--decline'], dest='decline', nargs=None, const=None, default=30, type=<class 'int'>, choices=None, required=False, help='number of epochs to decline', metavar=None)

In [19]:
# 设置模型参数
args = parser.parse_args(['--epochs', '1000', '--batch_size', '8', 
                            '--device', 'cuda', '--lr', '0.001', 
                            '--rat', '0.9', '--decline', '20'])
# 执行训练
model = train(args, x_train, x_mask, x_edge_df)

KeyboardInterrupt: 

# 重新训练并进行预测

In [ ]:
# 将模型在整个数据集进行训练
args = parser.parse_args(['--epochs', '1000', '--batch_size', '16', 
                            '--device', 'cuda', '--lr', '0.001', 
                            '--rat', '1.0', '--decline', '20'])
final_model = train(args, x_train, x_mask, x_edge_df)

In [ ]:
# 使用当前的时间作为文件名来保存模型
import time
t = time.localtime()
torch.save(model.state_dict(), 'model/model_{}_{}_{}_{}.pth'.format(t.tm_mon, t.tm_mday, t.tm_hour, t.tm_min))

In [ ]:
# 执行数据处理，只需要执行一次，后面只需要读取数据即可
# process_test_data("data/node_test_4_A.csv", "data/edge_test_4_A.csv", geohash_df_dict, number_date)

In [ ]:
# 执行数据处理，只需要执行一次，后面只需要读取数据即可
# 合并训练集和测试集，按照第一维合并
# x_test = np.concatenate((x_train[:,:,:-2], np.load("test_npdata/new_data.npy")), axis=0)
# x_mask_test = np.concatenate((x_mask, np.load("test_npdata/x_mask.npy")), axis=0)
# x_edge_test = np.concatenate((x_edge_df, np.load("test_npdata/x_edge_df.npy")), axis=0)

# np.save("total_test_npdata/x_total_test.npy", x_test)
# np.save("total_test_npdata/x_total_mask_test.npy", x_mask_test)
# np.save("total_test_npdata/x_total_edge_test.npy", x_edge_test)

In [ ]:
# 读取数据
temp1 = np.load("test_npdata/new_data.npy")
temp2 = np.load("test_npdata/x_mask.npy")
temp3 = np.load("test_npdata/x_edge_df.npy")

# print("temp1.shape: ", temp1.shape)
# print("temp2.shape: ", temp2.shape)
# print("temp3.shape: ", temp3.shape)

# print(temp1)
# print(temp2)
# print(temp3)

In [ ]:
# 用于检验原来数据处理代码中的bug
for i in range(4):
    for j in range(1140):
        if temp1[i][j][0] != 93.0:
            print('xxx')

In [ ]:
# 读取数据
x_test = np.load("total_test_npdata/x_total_test.npy")
x_mask_test = np.load("total_test_npdata/x_total_mask_test.npy")
x_edge_test = np.load("total_test_npdata/x_total_edge_test.npy")

# print("x_test.shape: ", x_test.shape)
# print("x_mask_test.shape: ", x_mask_test.shape)
# print("x_edge_test.shape: ", x_edge_test.shape)

# print(x_test)
# print(x_mask_test)
# print(x_edge_test)

In [ ]:
# 执行预测
predict(final_model, x_test, x_mask_test, x_edge_test)
    
print("Done!")

In [ ]:
# 提取node_test_4_A.csv中的geohash_id和date_id，与prediction/predict.csv中的数据合并
# 读取数据
test_df = pd.read_csv("data/node_test_4_A.csv", encoding='utf-8')
predict_df = pd.read_csv("prediction/predict.csv", encoding='utf-8', header=None)

print("test_df.shape: ", test_df.shape)
print("predict_df.shape: ", predict_df.shape)

# 合并数据
predict_df = pd.concat([test_df.iloc[:, 0], predict_df.iloc[:, 1], predict_df.iloc[:, 0], test_df.iloc[:, 1]], axis=1)

# 添加列名
predict_df.columns = ["geohash_id", "consumption_level", "activity_level", "date_id"] 

# 保存数据并以当前日期命名

# 今日提交次数
count = 2

# 读取当前日期
import datetime
now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d")

# 保存数据
predict_df.to_csv("submitCSV/submit_" + now + "_" + str(count) +".csv", index=False, header=True)

print("Done!")


In [ ]:
# 将文件中的所有逗号替换为tab
import os
import re

# 读取文件夹中的所有文件
path = "submitCSV/"
files = os.listdir(path)

# 逐个文件进行处理
for file in files:
    # 读取文件
    with open(path + file, "r", encoding="utf-8") as f:
        data = f.read()
    # 替换所有逗号
    data = re.sub(",", "\t", data)
    # 保存文件
    with open(path + file, "w", encoding="utf-8") as f:
        f.write(data)

print("Done!")